In [1]:
from Bio import Entrez, SeqIO
from Bio.Seq import Seq

from Bio.SeqRecord import SeqRecord


import vcf
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import functools

import pysam
import pandas as pd
import numpy as np

Entrez.email = "simon.burgermeister@gmail.com"


## Install with:
<br>
conda install -c bioconda bwa
<br>

## Build index:
<br>
<br>

In [3]:
!bwa index sequences4/ref/ref_sequence.fasta

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index sequences4/ref/ref_sequence.fasta
[main] Real time: 0.014 sec; CPU: 0.011 sec


In [4]:
!bwa mem sequences4/ref/ref_sequence.fasta sequences4/all.fasta > sequences4/aln.sam 

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 11 sequences (327935 bp)...
[M::mem_process_seqs] Processed 11 reads in 0.272 CPU sec, 0.272 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem sequences4/ref/ref_sequence.fasta sequences4/all.fasta
[main] Real time: 0.274 sec; CPU: 0.273 sec


In [5]:
!samtools view -S -b sequences4/aln.sam > sequences4/aligned.bam
#!samtools view -1 sequences4/aln.sam > sequences3/aln.bam
!samtools sort sequences4/aligned.bam -o sequences4/sorted.bam
!samtools index sequences4/sorted.bam

In [6]:
!samtools mpileup -uf sequences4/ref/ref_sequence.fasta sequences4/sorted.bam | bcftools call -mv  --multiallelic-caller --variants-only  > sequences4/var.raw.vcf

Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


In [7]:
f = vcf.Reader(filename='sequences4/var.raw.vcf')

my_type = defaultdict(int)
num_alts = defaultdict(int)

for rec in f:
    #print(rec.var_type)

    my_type[rec.var_type, rec.var_subtype] += 1
    if rec.is_snp:
        num_alts[len(rec.ALT)] += 1
print(my_type)
print(num_alts)

defaultdict(<class 'int'>, {('snp', 'ts'): 21, ('snp', 'tv'): 15, ('indel', 'del'): 4})
defaultdict(<class 'int'>, {1: 36})


In [9]:
import re

#samfile = pysam.AlignmentFile("sequences/sorted.bam", "rb")
samfile = pysam.AlignmentFile("sequences4/aligned.bam", "rb")
jj=0
for read in samfile:
    read_seq=str(read.seq)
    
    tt=str(read)
    rr=tt.split('\t0\t0')
    read_ID=rr[0]
    
    cig=read.cigarstring
    #print(read.cigarstring)
        
    match = re.findall(r'(\d+)(\w)',cig)
    
    s_start=read.reference_start
    print(read_ID)
    print(read_seq[100-s_start:120-s_start])
    print(read.cigarstring)
    print(' ')
    
    

NC_045512.2
GGCTGCATGCTTAGTGCACT
29903M
 
MW911846.1
GGCTGCATGCTTAGTGCACT
11257M9D10468M6D220M3D6277M1D1588M
 
MW911847.1
GGCTGCATGCTTAGTGCACT
11257M9D10468M6D220M3D6277M1D1590M
 
MW911849.1
GGCTGCATGCTTAGTGCACT
11257M9D10468M6D220M3D6277M1D1590M
 
MW911850.1
GGCTGCATGCTTAGTGCACT
11256M9D10468M6D220M3D6277M1D1586M
 
MW911860.1
GGCTGCATGCTTAGTGCACT
11257M9D10468M6D220M3D6277M1D1586M
 
MW911870.1
GGCTGCATGCTTAGTGCACT
11257M9D10468M6D220M3D6277M1D1590M
 
MW912051.1
GGCTGCATGCTTAGTGCACT
11257M9D10468M6D220M3D6277M1D1590M
 
MW913363.1
GGCTGCATGCTTAGTGCACT
11233M9D10468M6D220M3D6277M1D1565M
 
MW593784.1
GGCTGCATGCTTAGTGCACT
29798M
 
MW593785.1
GGCTGCATGCTTAGTGCACT
29798M
 
